# Real Choropleths

In the second part of the workshop, we will cover how to process data to generate the choropleths, including file formats, spatial reference systems, table joins and color stops.

## File Formats

In [ ]:
geojson
shapefile
kml
how to transform between

load zip code
load bbl buildings

## Spatial Reference Systems

In [ ]:
how to transform between
how to id which srs

## Table Joins

In [ ]:
how to join

## Color Stops

In [ ]:
how to determine color stop max min and in betweens